In [1]:
# Imports
import pandas as pd  # type: ignore
import numpy as np  # type: ignore
import os
import base64
from utils import (
    get_rules_list,
    create_train_test_split,
    create_model,
    create_adv_train_test_split,
    test_evasion,
    log,
)
from modsec import init_modsec

from sklearn.ensemble import RandomForestClassifier  # type: ignore
from wafamole.evasion import EvasionEngine  # type: ignore

log("Starting...")
rule_ids = get_rules_list()
modsec = init_modsec()

Starting...


In [2]:
# Set up variables

attack_data_path = "data/raw/attacks_20k.sql" # raw attack data
sane_data_path = "data/raw/sanes_20k.sql" # raw sane data
processed_data_path = "data/preprocessed/5"  # path to store the preprocessed train and test data

paranoia_level = 4

train_attacks_size = 500  # paper uses 10000
train_sanes_size = 500  # paper uses 10000
test_attacks_size = 200  # paper uses 2000
test_sanes_size = 200  # paper uses 2000

train_adv_size = 20  # paper uses 5000 (1/4 of total train set size)
test_adv_size = 10  # paper uses 2000 (1/2 of total test set size)

engine_settings = {
    "max_rounds": 200,
    "round_size": 10,
    "timeout": 10,
}

model = RandomForestClassifier(n_estimators=160, random_state=666)
model_adv = RandomForestClassifier(n_estimators=160, random_state=666)

In [3]:
# # Create train and test sets and train model
if not os.path.exists(processed_data_path):
    os.makedirs(processed_data_path)
train, test = create_train_test_split(
    attack_file=attack_data_path,
    sane_file=sane_data_path,
    train_attacks_size=train_attacks_size,
    train_sanes_size=train_sanes_size,
    test_attacks_size=test_attacks_size,
    test_sanes_size=test_sanes_size,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)
train.to_csv(f"{processed_data_path}/train_{train_attacks_size + train_sanes_size}.csv", index=False)
test.to_csv(f"{processed_data_path}/test_{test_attacks_size + test_sanes_size}.csv", index=False)
log("Train and test sets created", True)

# # load the train and test sets from disk
# train = pd.read_csv(f"{processed_data_path}/train_{train_attacks_size + train_sanes_size}.csv")
# test = pd.read_csv(f"{processed_data_path}/test_{test_attacks_size + test_sanes_size}.csv")
# train['vector'] = train['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
# test['vector'] = test['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

wafamole_model, threshold = create_model(
    train=train,
    test=test,
    model=model,
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Reading and parsing data...


Splitting into train and test...
Creating vectors...


Processing payloads: 100%|██████████| 400/400 [00:13<00:00, 30.61it/s]


Done!
Train shape: (1000, 3) | Test shape: (400, 3)
Train and test sets created
Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.98      0.94      0.96       200
        sane       0.95      0.98      0.96       200

    accuracy                           0.96       400
   macro avg       0.96      0.96      0.96       400
weighted avg       0.96      0.96      0.96       400

Adjusted threshold: 0.8664745283311458
              precision    recall  f1-score   support

      attack       0.82      0.99      0.90       200
        sane       0.99      0.79      0.87       200

    accuracy                           0.89       400
   macro avg       0.90      0.89      0.89       400
weighted avg       0.90      0.89      0.89       400



In [4]:
# adversarial training

engine = EvasionEngine(wafamole_model)
train_adv, test_adv = create_adv_train_test_split(
    train=train,
    test=test,
    train_adv_size=train_adv_size,
    test_adv_size=test_adv_size,
    engine=engine,
    engine_settings={
        **engine_settings,
        "threshold": threshold,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)
train_adv.to_csv(f"{processed_data_path}/train_adv_{train_adv_size}.csv", index=False)
test_adv.to_csv(f"{processed_data_path}/test_adv_{test_adv_size}.csv", index=False)
log("Adversarial train and test sets created", True)

# # load the train_adv and test_adv sets from disk
# train_adv = pd.read_csv(f"{processed_data_path}/train_adv_{train_adv_size}.csv")
# test_adv = pd.read_csv(f"{processed_data_path}/test_adv_{test_adv_size}.csv")
# train_adv['vector'] = train_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
# test_adv['vector'] = test_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

# train new model with train + train_adv
wafamole_model_adv, threshold_adv = create_model(
    train=pd.concat([train, train_adv]).sample(frac=1).reset_index(drop=True),
    test=pd.concat([test, test_adv]).sample(frac=1).reset_index(drop=True),
    model=model_adv,
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Optimizing payloads...


100%|██████████| 10/10 [00:46<00:00,  4.63s/it]


Train_adv shape: (20, 2) | Test_adv shape: (10, 2)
Creating vectors...


Processing payloads: 100%|██████████| 10/10 [00:00<00:00, 14.53it/s]


Adversarial train and test sets created
Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.98      0.93      0.96       210
        sane       0.93      0.98      0.96       200

    accuracy                           0.96       410
   macro avg       0.96      0.96      0.96       410
weighted avg       0.96      0.96      0.96       410

Adjusted threshold: 0.85
              precision    recall  f1-score   support

      attack       0.84      0.99      0.91       210
        sane       0.99      0.80      0.88       200

    accuracy                           0.90       410
   macro avg       0.91      0.89      0.89       410
weighted avg       0.91      0.90      0.89       410



In [5]:
# Test the model (without adversarial training)
test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=wafamole_model,
    engine=EvasionEngine(wafamole_model),
    engine_eval_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 60,
        "threshold": 0.0,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.9487
[!] Execution timed out
Reached confidence 0.7407444258208398
with payload
'Select\xa0SLEEP((SELECT 5))#";^'
Min payload: b'Select\xc2\xa0SLEEP((SELECT 5))#";^'
Min confidence: 0.74074
Reduced confidence from 0.9487 to 0.74074 (reduction of 0.20796)

Evasion successful


In [6]:
# Test the model (with adversarial training)
test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=wafamole_model_adv,
    engine=EvasionEngine(wafamole_model_adv),
    engine_eval_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 60,
        "threshold": 0.0,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.93597
[!] Execution timed out
Reached confidence 0.6525297002620807
with payload
'SeLECT\x0bSLEEP((seLECT\t0x5))#";2/3^9'
Min payload: b'SeLECT\x0bSLEEP((seLECT\t0x5))#";2/3^9'
Min confidence: 0.65253
Reduced confidence from 0.93597 to 0.65253 (reduction of 0.28344)

Evasion successful
